In [1]:
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args

    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)

    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Example: US States Data

Merge and join operations come up most often when combining data from different sources.
Here we will consider an example of some data about US states and their populations.
The data files can be found at [http://github.com/jakevdp/data-USstates](http://github.com/jakevdp/data-USstates):

Let's take a look at the three datasets, using the Pandas ``read_csv`` function:

In [2]:
pop = pd.read_csv("./files/state-population.csv")
areas =pd.read_csv("./files/state-areas.csv")
abbrevs =pd.read_csv("./files/state-abbrevs.csv")

display('pop.head()', 'areas.head()', 'abbrevs.head()')

pop.head()
  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

areas.head()
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

abbrevs.head()
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

Given this information, say we want to compute a relatively straightforward result: rank US states and territories by their 2010 population density.
We clearly have the data here to find this result, but we'll have to combine the datasets to do so.

We'll start with a many-to-one merge that will give us the full state names within the population `DataFrame`.
We want to merge based on the `state/region`  column of `pop` and the `abbreviation` column of `abbrevs`.
We'll use `how='outer'` to make sure no data is thrown away due to mismatched labels:

In [3]:
p1=pd.DataFrame(pop)
ar1=pd.DataFrame(areas)
ab1=pd.DataFrame(abbrevs)


In [18]:
merged =pd.merge(p1,ab1,left_on='state/region',right_on='abbreviation',how='outer')
merged = merged.drop(columns='abbreviation',axis=1)                                 # drop column having duplicate info after merging
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [14]:
merged.duplicated(keep='first')
merged

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alabama,AL
2,AL,under18,2010,1130966.0,Alabama,AL
3,AL,total,2010,4785570.0,Alabama,AL
4,AL,under18,2011,1125763.0,Alabama,AL
...,...,...,...,...,...,...
2539,USA,total,2010,309326295.0,NaN,NaN
2540,USA,under18,2011,73902222.0,NaN,NaN
2541,USA,total,2011,311582564.0,NaN,NaN
2542,USA,under18,2012,73708179.0,NaN,NaN


Let's double-check whether there were any mismatches here, which we can do by looking for rows with nulls:

In [19]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

Some of the ``population`` values are null; let's figure out which these are!

In [20]:
ans=merged[merged['population'].isnull()]
ans

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN
2453,PR,under18,1993,NaN,NaN
2454,PR,under18,1992,NaN,NaN
2455,PR,total,1992,NaN,NaN
2456,PR,under18,1994,NaN,NaN
2457,PR,total,1994,NaN,NaN


It appears that all the null population values are from Puerto Rico prior to the year 2000; this is likely due to this data not being available in the original source.

More importantly, we see that some of the new `state` entries are also null, which means that there was no corresponding entry in the `abbrevs` key!
Let's figure out which regions lack this match:

In [24]:
merged.loc[merged['state'].isnull()]

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN
...,...,...,...,...,...
2539,USA,total,2010,309326295.0,NaN
2540,USA,under18,2011,73902222.0,NaN
2541,USA,total,2011,311582564.0,NaN
2542,USA,under18,2012,73708179.0,NaN


In [29]:
merged.loc[merged['state'].isnull()]

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN
...,...,...,...,...,...
2539,USA,total,2010,309326295.0,NaN
2540,USA,under18,2011,73902222.0,NaN
2541,USA,total,2011,311582564.0,NaN
2542,USA,under18,2012,73708179.0,NaN


We can quickly infer the issue: our population data includes entries for Puerto Rico (PR) and the United States as a whole (USA), while these entries do not appear in the state abbreviation key.
We can fix these quickly by filling in appropriate entries:

In [25]:
merged.loc[merged['state/region'] == 'PR', 'state']
merged.loc[merged['state/region'] == 'USA', 'state']

2496    NaN
2497    NaN
2498    NaN
2499    NaN
2500    NaN
2501    NaN
2502    NaN
2503    NaN
2504    NaN
2505    NaN
2506    NaN
2507    NaN
2508    NaN
2509    NaN
2510    NaN
2511    NaN
2512    NaN
2513    NaN
2514    NaN
2515    NaN
2516    NaN
2517    NaN
2518    NaN
2519    NaN
2520    NaN
2521    NaN
2522    NaN
2523    NaN
2524    NaN
2525    NaN
2526    NaN
2527    NaN
2528    NaN
2529    NaN
2530    NaN
2531    NaN
2532    NaN
2533    NaN
2534    NaN
2535    NaN
2536    NaN
2537    NaN
2538    NaN
2539    NaN
2540    NaN
2541    NaN
2542    NaN
2543    NaN
Name: state, dtype: object

In [26]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

No more nulls in the `state` column: we're all set!

Now we can merge the result with the area data using a similar procedure.
Examining our results, we will want to join on the `state` column in both:

In [32]:
final =pd.merge(merged,ar1,on='state',how='outer')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012.0,1117489.0,Alabama,52423.0
1,AL,total,2012.0,4817528.0,Alabama,52423.0
2,AL,under18,2010.0,1130966.0,Alabama,52423.0
3,AL,total,2010.0,4785570.0,Alabama,52423.0
4,AL,under18,2011.0,1125763.0,Alabama,52423.0


Again, let's check for nulls to see if there were any mismatches:

In [33]:
final.isnull().any()

state/region     True
ages             True
year             True
population       True
state            True
area (sq. mi)    True
dtype: bool

There are nulls in the ``area`` column; we can take a look to see which regions were ignored here:

In [36]:
final.loc[final['area (sq. mi)'].isnull()]

,state/region,ages,year,population,state,area (sq. mi)
2448,PR,under18,1990.0,NaN,NaN,NaN
2449,PR,total,1990.0,NaN,NaN,NaN
2450,PR,total,1991.0,NaN,NaN,NaN
2451,PR,under18,1991.0,NaN,NaN,NaN
2452,PR,total,1993.0,NaN,NaN,NaN
...,...,...,...,...,...,...
2539,USA,total,2010.0,309326295.0,NaN,NaN
2540,USA,under18,2011.0,73902222.0,NaN,NaN
2541,USA,total,2011.0,311582564.0,NaN,NaN
2542,USA,under18,2012.0,73708179.0,NaN,NaN


We see that our ``areas`` ``DataFrame`` does not contain the area of the United States as a whole.
We could insert the appropriate value (using the sum of all state areas, for instance), but in this case we'll just drop the null values because the population density of the entire United States is not relevant to our current discussion:

In [37]:
final['state'][final['area (sq. mi)'].isnull()].fillna(final['area (sq. mi)'].sum())

2448    181773947.0
2449    181773947.0
2450    181773947.0
2451    181773947.0
2452    181773947.0
           ...     
2539    181773947.0
2540    181773947.0
2541    181773947.0
2542    181773947.0
2543    181773947.0
Name: state, Length: 96, dtype: float64

In [45]:
final.dropna(inplace=True)
final

,state/region,ages,year,population,state,area (sq. mi)
state,,,,,,
Alabama,AL,under18,2012.0,1117489.0,Alabama,52423.0
Alabama,AL,total,2012.0,4817528.0,Alabama,52423.0
Alabama,AL,under18,2010.0,1130966.0,Alabama,52423.0
Alabama,AL,total,2010.0,4785570.0,Alabama,52423.0
Alabama,AL,under18,2011.0,1125763.0,Alabama,52423.0
...,...,...,...,...,...,...
Wyoming,WY,under18,1993.0,137458.0,Wyoming,97818.0
Wyoming,WY,total,1991.0,459260.0,Wyoming,97818.0
Wyoming,WY,under18,1991.0,136720.0,Wyoming,97818.0


Now we have all the data we need. To answer the question of interest, let's first select the portion of the data corresponding with the year 2010, and the total population.
We'll use the `query` function to do this quickly (this requires the NumExpr package to be installed; see [High-Performance Pandas: `eval()` and `query()`](03.12-Performance-Eval-and-Query.ipynb)):

In [51]:
data2=final.query('year==2010 and ages="total"')
data2.head()

SyntaxError: Python keyword not valid identifier in numexpr query (<unknown>, line 1)

Now let's compute the population density and display it in order.
We'll start by re-indexing our data on the state, and then compute the result:

In [40]:
final.sort_values(by='population')

,state/region,ages,year,population,state,area (sq. mi)
388,DC,under18,2010.0,101309.0,District of Columbia,68.0
393,DC,under18,2009.0,102098.0,District of Columbia,68.0
397,DC,under18,2008.0,102257.0,District of Columbia,68.0
386,DC,under18,2011.0,103906.0,District of Columbia,68.0
395,DC,under18,2007.0,104126.0,District of Columbia,68.0
...,...,...,...,...,...,...
200,CA,total,2009.0,36961229.0,California,163707.0
197,CA,total,2010.0,37333601.0,California,163707.0
195,CA,total,2011.0,37668681.0,California,163707.0
193,CA,total,2012.0,37999878.0,California,163707.0


In [44]:
new_idx=final['state']
final.set_index(new_idx,inplace=True)

The result is a ranking of US states, plus Washington, DC, and Puerto Rico, in order of their 2010 population density, in residents per square mile.
We can see that by far the densest region in this dataset is Washington, DC (i.e., the District of Columbia); among states, the densest is New Jersey.

We can also check the end of the list:

We see that the least dense state, by far, is Alaska, averaging slightly over one resident per square mile.

This type of data merging is a common task when trying to answer questions using real-world data sources.
I hope that this example has given you an idea of some of the ways you can combine the tools we've covered in order to gain insight from your data!